# **FOREWORD**

Thanks to the baseline work [here](https://www.kaggle.com/code/masayakawamata/s5e11-te-xgb-interaction-features). I add a couple of ML models to the original work and complete my baseline work

This is my starter work for the November Playground 2025 challenge using the train and original as rows and interaction features with mean and count encoders. <br>I also add a few quantile binner features for numeric columns from [here](https://www.kaggle.com/code/yousefelshahat2/simple-xgboost-only-competition-data-s5e11/notebook)

# **IMPORTS**

In [ ]:
import warnings, torch
import pandas as pd, numpy as np
warnings.simplefilter('ignore')
from itertools import combinations
from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold, KFold
from xgboost import XGBClassifier as XGBC
from lightgbm import LGBMClassifier as LGBMC, log_evaluation, early_stopping
from catboost import CatBoostClassifier as CBC
from sklearn.metrics import *

from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
test_req = False

if test_req :
    print("THIS IS A SYNTAX CHECK RUN")
    nest = 150
    es   = 50
else:
    nest = 12000
    es   = 300

# **PREPROCESSING**

In [ ]:

train = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')
orig = pd.read_csv('/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv')

print('Train Shape:', train.shape)
print('Test Shape:', test.shape)
print('Orig Shape:', orig.shape)

TARGET = 'loan_paid_back'
CATS   = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
BASE   = [col for col in train.columns if col not in ['id', TARGET]]
NUMS   = list(set(BASE) - set(CATS))

# **FEATURE ENGINEERING**

- Original as columns
- Combination interactions
- k-bins discretization

In [ ]:

INTER = []

for col1, col2 in tqdm(combinations(BASE, 2)):
    new_col_name = f'{col1}_{col2}'
    INTER.append(new_col_name)
    for df in [train, test, orig] :
        df[new_col_name] = df[col1].astype(str) + '_' + df[col2].astype(str)
        
print(f'{len(INTER)} Features')

for col1, col2, col3 in combinations(CATS, 3 ):
    new_col_name = f'{col1}_{col2}_{col3}'
    INTER.append(new_col_name)
    for df in [train, test, orig]:
        df[new_col_name] = df[col1].astype(str) + '_' + df[col2].astype(str) + df[col3].astype(str)
        
print(f'{len(INTER)} Features')

ORIG = []

for col in BASE:
    # MEAN
    mean_map = orig.groupby(col)[TARGET].mean()
    new_mean_col_name = f"orig_mean_{col}"
    mean_map.name = new_mean_col_name
    
    train = train.merge(mean_map, on=col, how='left')
    test = test.merge(mean_map, on=col, how='left')
    ORIG.append(new_mean_col_name)

    # COUNT
    new_count_col_name = f"orig_count_{col}"
    count_map = orig.groupby(col).size().reset_index(name=new_count_col_name)
    
    train = train.merge(count_map, on=col, how='left')
    test  = test.merge(count_map, on=col, how='left')
    ORIG.append(new_count_col_name)

print(len(ORIG), 'Orig Features Created!!')

BINS = []
for col in NUMS :
    for q in [5, 10, 15, 20, 25]:
        try:
            train_bins, bins  = pd.qcut(
                df[col], q=q, labels=False, retbins=True, duplicates="drop"
            )
            train[f"{col}_bin{q}"] = train_bins
            
            test[f"{col}_bin{q}"] = pd.cut(
                df_test[col], bins=bins, labels=False, include_lowest=True
            )
            
        except Exception:
            train[f"{col}_bin{q}"] = 0
            test[f"{col}_bin{q}"] = 0

        BINS.append(f"{col}_bin{q}")

print(len(BINS), 'Discretization Features Created!!')

In [ ]:
FEATURES = BASE + ORIG + INTER + BINS
print(len(FEATURES), 'Features.')

X = train[FEATURES]
y = train[TARGET]

with np.printoptions(linewidth = 150, threshold = 10000):
    print(f"\n\n---> Selected features\n\n")
    print(np.array(FEATURES))
    print()

# **MODEL TRAINING**

In [ ]:

N_SPLITS = 5
skf      = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)


Mdl_Master = \
{     
 f'XGB1C'  : [
              XGBC(**{ "objective"            : "binary:logistic",
                       "eval_metric"          : "auc",
                       'device'               : "cuda:0" if torch.cuda.is_available() else "cpu",
                       'learning_rate'        : 0.005,
                       'n_estimators'         : nest,
                       'max_depth'            : 7,
                       'subsample'            : 0.90,
                       'colsample_bytree'     : 0.55,
                       'reg_lambda'           : 1.85,
                       'reg_alpha'            : 0.01,
                       'verbosity'            : 0,
                       'random_state'         : 42,
                       'enable_categorical'   : True,
                       'early_stopping_rounds' : es,
                      } 
                   ),
              {"verbose" : 0}
             ],
}




In [ ]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    """
    Target Encoder that supports multiple aggregation functions,
    internal cross-validation for leakage prevention, and smoothing.

    Parameters
    ----------
    cols_to_encode : list of str
        List of column names to be target encoded.

    aggs : list of str, default=['mean']
        List of aggregation functions to apply. Any function accepted by
        pandas' `.agg()` method is supported, such as:
        'mean', 'std', 'var', 'min', 'max', 'skew', 'nunique', 
        'count', 'sum', 'median'.
        Smoothing is applied only to the 'mean' aggregation.

    cv : int, default=5
        Number of folds for cross-validation in fit_transform.

    smooth : float or 'auto', default='auto'
        The smoothing parameter `m`. A larger value puts more weight on the 
        global mean. If 'auto', an empirical Bayes estimate is used.
        
    drop_original : bool, default=False
        If True, the original columns to be encoded are dropped.
    """
    def __init__(self, cols_to_encode, aggs=['mean'], cv=5, smooth='auto', drop_original=False):
        self.cols_to_encode = cols_to_encode
        self.aggs = aggs
        self.cv = cv
        self.smooth = smooth
        self.drop_original = drop_original
        self.mappings_ = {}
        self.global_stats_ = {}

    def fit(self, X, y):
        """
        Learn mappings from the entire dataset.
        These mappings are used for the transform method on validation/test data.
        """
        temp_df = X.copy()
        temp_df['target'] = y

        # Learn global statistics for each aggregation
        for agg_func in self.aggs:
            self.global_stats_[agg_func] = y.agg(agg_func)

        # Learn category-specific mappings
        for col in self.cols_to_encode:
            self.mappings_[col] = {}
            for agg_func in self.aggs:
                mapping = temp_df.groupby(col)['target'].agg(agg_func)
                self.mappings_[col][agg_func] = mapping
        
        return self

    def transform(self, X):
        """
        Apply learned mappings to the data.
        Unseen categories are filled with global statistics.
        """
        X_transformed = X.copy()
        for col in self.cols_to_encode:
            for agg_func in self.aggs:
                new_col_name = f'TE_{col}_{agg_func}'
                map_series = self.mappings_[col][agg_func]
                X_transformed[new_col_name] = X[col].map(map_series)
                X_transformed[new_col_name].fillna(self.global_stats_[agg_func], inplace=True)
        
        if self.drop_original:
            X_transformed.drop(columns=self.cols_to_encode, inplace=True)
            
        return X_transformed

    def fit_transform(self, X, y):
        """
        Fit and transform the data using internal cross-validation to prevent leakage.
        """
        # First, fit on the entire dataset to get global mappings for transform method
        self.fit(X, y)

        # Initialize an empty DataFrame to store encoded features
        encoded_features = pd.DataFrame(index=X.index)
        
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=42)

        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val = X.iloc[val_idx]
            
            temp_df_train = X_train.copy()
            temp_df_train['target'] = y_train

            for col in self.cols_to_encode:
                # --- Calculate mappings only on the training part of the fold ---
                for agg_func in self.aggs:
                    new_col_name = f'TE_{col}_{agg_func}'
                    
                    # Calculate global stat for this fold
                    fold_global_stat = y_train.agg(agg_func)
                    
                    # Calculate category stats for this fold
                    mapping = temp_df_train.groupby(col)['target'].agg(agg_func)

                    # --- Apply smoothing only for 'mean' aggregation ---
                    if agg_func == 'mean':
                        counts = temp_df_train.groupby(col)['target'].count()
                        
                        m = self.smooth
                        if self.smooth == 'auto':
                            # Empirical Bayes smoothing
                            variance_between = mapping.var()
                            avg_variance_within = temp_df_train.groupby(col)['target'].var().mean()
                            if variance_between > 0:
                                m = avg_variance_within / variance_between
                            else:
                                m = 0  # No smoothing if no variance between groups
                        
                        # Apply smoothing formula
                        smoothed_mapping = (counts * mapping + m * fold_global_stat) / (counts + m)
                        encoded_values = X_val[col].map(smoothed_mapping)
                    else:
                        encoded_values = X_val[col].map(mapping)
                    
                    # Store encoded values for the validation fold
                    encoded_features.loc[X_val.index, new_col_name] = encoded_values.fillna(fold_global_stat)

        # Merge with original DataFrame
        X_transformed = X.copy()
        for col in encoded_features.columns:
            X_transformed[col] = encoded_features[col]
            
        if self.drop_original:
            X_transformed.drop(columns=self.cols_to_encode, inplace=True)
            
        return X_transformed

In [ ]:

OOF_Preds    = []
Mdl_Preds    = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    
    print(f'--- Fold {fold}/{N_SPLITS} ---')
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    X_test         = test[FEATURES].copy()

    TE      = TargetEncoder(
        cols_to_encode=INTER, 
        cv=5, 
        smooth='auto', 
        aggs=['mean'], 
        drop_original=True
    )
    
    X_train = TE.fit_transform(X_train, y_train)
    X_val   = TE.transform(X_val)
    X_test  = TE.transform(X_test)

    X_train[CATS] = X_train[CATS].astype('category')
    X_val[CATS]   = X_val[CATS].astype('category')
    X_test[CATS]  = X_test[CATS].astype('category')

    oof_preds , test_preds = [], []
    
    for method, (model, fit_params) in Mdl_Master.items():
  
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], **fit_params)
        val_preds = pd.Series(model.predict_proba(X_val)[:, 1], name = method, index = val_idx)
        oof_preds.append(val_preds)
        test_preds.append(pd.Series( model.predict_proba(X_test)[:, 1], name = method))

        print(f"---> Model {method} fitted successfully")

    oof_preds  = pd.concat(oof_preds, axis=1)
    test_preds = pd.concat(test_preds, axis=1) 

    OOF_Preds.append(oof_preds)
    Mdl_Preds.append(test_preds)

OOF_Preds = pd.concat(OOF_Preds, axis=0, ignore_index = False).sort_index(ascending = True)
Mdl_Preds = pd.concat(Mdl_Preds, axis=0, ignore_index = False).groupby(level = 0).mean()

# **ENSEMBLE**

Let's do a simple average for now and then submit. Ensembling is a key step for Playground competitions and this step needs to be improved

In [ ]:
for col in OOF_Preds.columns :
    score  = roc_auc_score(y, OOF_Preds[col])
    print(f"---> Score = {score:,.8f} | {col}")

oof_preds = np.mean(OOF_Preds.values,  axis=1)
score     = roc_auc_score(y, oof_preds)
print(f"\n---> Average Ensemble Score = {score:,.8f}\n")

test_preds = Mdl_Preds.mean(axis = 1).values

sub_fl = pd.read_csv(
    f"/kaggle/input/playground-series-s5e11/sample_submission.csv",
    index_col = "id",
)

# Blending with same CV scheme public notebook
sub_public = pd.read_csv(
    f"/kaggle/input/ps5e11-agentic-ai-solution-single-xgb/submission_4.csv"
)["loan_paid_back"].values.flatten()

sub_fl["loan_paid_back"] = test_preds * 0.20 + 0.80 * sub_public

sub_fl.to_csv(f"submission.csv", index = True)

print()
!head submission.csv